Jupyter Notebook to visualize our slurm files and other csv benchmarking files

In [28]:
!ls results

ghc         normal_psc  old         uniform_psc


In [30]:
import glob
import pandas as pd
import numpy as np

In [31]:
slurm_fns = glob.glob("results/normal_psc/*"); len(slurm_fns)

4

In [32]:
psc_header = "sync_time,fine_lock_time,lock_free_time,num_threads,update_prob,removal_prob,variance,array_length".split(',')
psc_header

['sync_time',
 'fine_lock_time',
 'lock_free_time',
 'num_threads',
 'update_prob',
 'removal_prob',
 'variance',
 'array_length']

In [33]:
i = 1

In [34]:
with open(slurm_fns[i]) as f:
    slurm = f.read()
slurm.split()

['7.03154,8.97655,8.54301,1,0.4,0.4,100000,10000000',
 '19.3239,4.68326,5.13739,4,0.4,0.4,100000,10000000',
 '26.7916,2.21478,2.10858,16,0.4,0.4,100000,10000000',
 '31.2271,2.03605,2.15236,64,0.4,0.4,100000,10000000',
 '35.8169,1.55549,1.3027,128,0.4,0.4,100000,10000000']

In [35]:
df_all = None

In [36]:
df = pd.read_csv(slurm_fns[i],names=psc_header)
df["dist"] = "normal"
df["dist_param1"] = 0.0 # mean of 0
df = df.rename(columns={"variance":"dist_param2"})
df

,sync_time,fine_lock_time,lock_free_time,num_threads,update_prob,removal_prob,dist_param2,array_length,dist,dist_param1
0,7.03154,8.97655,8.54301,1,0.4,0.4,100000,10000000,normal,0.0
1,19.32390,4.68326,5.13739,4,0.4,0.4,100000,10000000,normal,0.0
2,26.79160,2.21478,2.10858,16,0.4,0.4,100000,10000000,normal,0.0
3,31.22710,2.03605,2.15236,64,0.4,0.4,100000,10000000,normal,0.0
4,35.81690,1.55549,1.30270,128,0.4,0.4,100000,10000000,normal,0.0


In [40]:
df_all = None
for i in range(len(slurm_fns)):
    with open(slurm_fns[i]) as f:
        slurm = f.read()
    slurm.split()
    df_new = pd.read_csv(slurm_fns[i],names=psc_header)
    df_new["dist"] = "normal"
    df_new["dist_param1"] = 0.0 # mean of 0
    df_new = df_new.rename(columns={"variance":"dist_param2","array_length":"array_len"})
    print("adding ", len(df_new), "rows")
    if df_all is None:
        df_all = df_new
    else:
        df_all = df_all.append(df_new, ignore_index=True)
slurm_uniform_fns = glob.glob("results/uniform_psc/*")

for i in range(len(slurm_uniform_fns)):
    with open(slurm_uniform_fns[i]) as f:
        slurm = f.read()
    slurm.split()
    df_new = pd.read_csv(slurm_uniform_fns[i],names=psc_header)
    df_new["dist"] = "uniform"
    df_new["dist_param1"] = -1 * df_new["variance"] # start of -variance
    df_new = df_new.rename(columns={"variance":"dist_param2","array_length":"array_len"})  # end of range
    print("adding ", len(df_new), "rows")
    if df_all is None:
        df_all = df_new
    else:
        df_all = df_all.append(df_new,ignore_index=True)
df_all["machine"] = "psc"
df_all.tail()

adding  5 rows
adding  5 rows
adding  5 rows
adding  5 rows
adding  30 rows


,sync_time,fine_lock_time,lock_free_time,num_threads,update_prob,removal_prob,dist_param2,array_len,dist,dist_param1,machine
45,0.388728,0.481089,0.388015,1,0.5,0.5,2000000,1000000,uniform,-2000000.0,psc
46,1.326030,0.287039,0.268720,4,0.5,0.5,2000000,1000000,uniform,-2000000.0,psc
47,1.812240,0.178643,0.175414,16,0.5,0.5,2000000,1000000,uniform,-2000000.0,psc
48,2.271500,0.104505,0.105888,64,0.5,0.5,2000000,1000000,uniform,-2000000.0,psc
49,2.781700,0.094824,0.095012,128,0.5,0.5,2000000,1000000,uniform,-2000000.0,psc


now, also make second df for uniform slurm files

## section of ghc benchmarking

In [41]:
ghc_header = "dist,dist_param1,dist_param2,sync_time,fine_lock_time,lock_free_time,num_threads,update_prob,removal_prob,array_len".split(",")
ghc_header

['dist',
 'dist_param1',
 'dist_param2',
 'sync_time',
 'fine_lock_time',
 'lock_free_time',
 'num_threads',
 'update_prob',
 'removal_prob',
 'array_len']

In [46]:
df = pd.read_csv("results/ghc/ghc_benchmark.csv"); df

,dist,dist_param1,dist_param2,sync_time,fine_lock_time,lock_free_time,num_threads,update_prob,removal_prob,array_len
0,normal,0.0,10000.0,0.044213,0.006083,0.006234,8,0.1,0.1,100000
1,uniform,-1000.0,1000.0,0.020491,0.004619,0.005491,8,0.1,0.1,100000
2,bimodal,0.0,2500.0,0.039531,0.004563,0.005075,8,0.1,0.1,100000


In [47]:
df["machine"] = "ghc"

In [48]:
df_both = df.append(df_all, ignore_index=True)

In [49]:
df_both

,dist,dist_param1,dist_param2,sync_time,fine_lock_time,lock_free_time,num_threads,update_prob,removal_prob,array_len,machine
0,normal,0.0,10000.0,0.044213,0.006083,0.006234,8,0.10,0.10,100000,ghc
1,uniform,-1000.0,1000.0,0.020491,0.004619,0.005491,8,0.10,0.10,100000,ghc
2,bimodal,0.0,2500.0,0.039531,0.004563,0.005075,8,0.10,0.10,100000,ghc
3,normal,0.0,1000.0,1.642920,1.858560,1.830590,1,0.10,0.10,10000000,psc
4,normal,0.0,1000.0,7.209420,1.883570,1.838910,4,0.10,0.10,10000000,psc
5,normal,0.0,1000.0,15.670500,2.015410,1.973570,16,0.10,0.10,10000000,psc
6,normal,0.0,1000.0,27.639600,1.190960,1.166340,64,0.10,0.10,10000000,psc
7,normal,0.0,1000.0,29.218400,0.781542,0.733583,128,0.10,0.10,10000000,psc
8,normal,0.0,100000.0,7.031540,8.976550,8.543010,1,0.40,0.40,10000000,psc
9,normal,0.0,100000.0,19.323900,4.683260,5.137390,4,0.40,0.40,10000000,psc


In [50]:
df_both.to_csv("scratch.csv")
with open("scratch.csv") as f:
    print(f.read())  # to copy into sheets

,dist,dist_param1,dist_param2,sync_time,fine_lock_time,lock_free_time,num_threads,update_prob,removal_prob,array_len,machine
0,normal,0.0,10000.0,0.044213,0.006083,0.006234,8,0.1,0.1,100000,ghc
1,uniform,-1000.0,1000.0,0.020491,0.004619,0.005491,8,0.1,0.1,100000,ghc
2,bimodal,0.0,2500.0,0.039531,0.004563,0.005075,8,0.1,0.1,100000,ghc
3,normal,0.0,1000.0,1.64292,1.85856,1.83059,1,0.1,0.1,10000000,psc
4,normal,0.0,1000.0,7.20942,1.88357,1.83891,4,0.1,0.1,10000000,psc
5,normal,0.0,1000.0,15.6705,2.01541,1.97357,16,0.1,0.1,10000000,psc
6,normal,0.0,1000.0,27.6396,1.19096,1.16634,64,0.1,0.1,10000000,psc
7,normal,0.0,1000.0,29.2184,0.781542,0.733583,128,0.1,0.1,10000000,psc
8,normal,0.0,100000.0,7.03154,8.97655,8.54301,1,0.4,0.4,10000000,psc
9,normal,0.0,100000.0,19.3239,4.68326,5.13739,4,0.4,0.4,10000000,psc
10,normal,0.0,100000.0,26.7916,2.21478,2.10858,16,0.4,0.4,10000000,psc
11,normal,0.0,100000.0,31.2271,2.03605,2.15236,64,0.4,0.4,10000000,psc
12,normal,0.0,100000.0,35.8169,1.55549,1.30